In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Project_Data/Review_data.csv', index_col=0)

In [11]:
np.histogram(df.reviewerID.value_counts().values,
              bins = [i for i in range(21)])

(array([     0, 203644,  39944,   8578,   7183,    776,    229,     62,
           103,     18,      8,      4,      4,      1,      1,      2,
             0,      0,      0,      1], dtype=int64),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20]))

In [1]:
import pandas as pd

metadata = pd.read_csv('../Project_Data/Metadata.csv', index_col=0)
asins = metadata['asin'].to_list()
image_ind = {}
for asin in asins:
    try:
        Image.open('../Project_Data/Images/'+ str(asin)+ '_0.jpg')
        image_ind[asin] = True
    except:
        image_ind[asin] = False

    break

In [3]:
from tqdm import tqdm

for ind, asin in tqdm(enumerate(asins)):
    try:
        Image.open('../Project_Data/Images/'+ str(asin)+ '_0.jpg')
        image_ind[asin] = True
    except:
        image_ind[asin] = False

    if ind>=1000:
        break

1000it [00:00, 999357.64it/s]


In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')



d:\Softwares\Installed\Anaconda\envs\isr_project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import torch

In [16]:
torch.cuda.is_available()

False